# Atari Breakout in RLlib

In [ ]:
#!pip install ray[rllib]
#!pip install ipywidgets gputil
#!pip install gym[atari] autorom[accept-rom-license] ray[rllib] torch torchvision

In [ ]:
import gym
import ray
from ray import tune

def main():
    ray.init(log_to_driver=False)
    
    config = {
        "env": "Breakout-v5",
        "num_workers": 1,  
        "framework": "torch",  
        "model": {
           "dim": 84,
            "conv_filters": [
                [32, [8, 8], 4],
                [64, [4, 4], 2],
                [64, [3, 3], 1]
            ],
            "conv_activation": "relu",
        },
        "lr": 0.0001,
        "gamma": 0.99,
        "train_batch_size": 5000,
        "rollout_fragment_length": "auto",
        "sgd_minibatch_size": 500,
        "num_sgd_iter": 10,
        "clip_param": 0.2,
        "entropy_coeff": 0.01,
    }

    stop_criteria = {
        "training_iteration": 200,
        "episode_reward_mean": 200,
    }

    results = tune.run(
        "PPO",
        config=config,
        stop=stop_criteria,
        checkpoint_at_end=True,
        checkpoint_freq=10
    )
    
    checkpoints = results.get_trial_checkpoints_paths(
        trial=results.get_best_trial("episode_reward_mean"),
        metric="episode_reward_mean"
    )
    
    print("Best checkpoint path:", checkpoints[0][0])

if __name__ == "__main__":
    main()